In [1]:
import keras

keras.__version__

'2.15.0'

In [1]:
from tensorflow.keras.utils import image_dataset_from_directory
from pathlib import Path
from config import DATASET
import tensorflow as tf

# Ensure dataset_path is a Path object
dataset_path = Path(DATASET)

train_dataset = image_dataset_from_directory(
    dataset_path / 'train',
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    seed=0,
    validation_split=0.2,
    subset='training'
)

validation_dataset = image_dataset_from_directory(
    dataset_path / 'train',
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    seed=0,
    validation_split=0.2,
    subset='validation'
)

# validation_split = 0.2
# total_batches = tf.data.experimental.cardinality(train_dataset).numpy()
# val_batches = int(validation_split * total_batches)

# train_dataset = train_dataset.skip(val_batches)
# validation_dataset = train_dataset.take(val_batches)


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [4]:
train_dataset

NameError: name 'train_dataset' is not defined

In [5]:
validation_dataset

NameError: name 'validation_dataset' is not defined

In [6]:
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.data import AUTOTUNE

train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)

# caching/prefetching for better performance
# train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

NameError: name 'train_dataset' is not defined

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)